In [5]:
from faker import Faker
import random
from datetime import datetime, timedelta
import numpy as np

# Initialize Faker
fake = Faker()

# Function to generate random date of birth within a given range
def generate_dob(start_date, end_date):
    dob = fake.date_between(start_date=start_date, end_date=end_date)
    return dob.strftime('%Y-%m-%d')

# Function to generate random age based on date of birth
def calculate_age(dob, mean_adult_age=40, std_dev_adult=15, child_prob=0.15):
    today = datetime.today()
    dob = datetime.strptime(dob, '%Y-%m-%d')
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    if random.random() < child_prob:
        age = np.random.randint(3, 13)  # Random age between 3 and 12 with probability 15%
    else:
        age = np.clip(np.round(np.random.normal(mean_adult_age, std_dev_adult)), 18, None)
    return age

# Function to generate random gender
def generate_gender():
    genders = ['Male', 'Female']
    return random.choice(genders)

# Function to generate patient data
def generate_patient_data(num_patients):
    patients = []
    for _ in range(num_patients):
        first_name = fake.first_name()
        last_name = fake.last_name()
        name = f"{first_name} {last_name}"       
        dob = generate_dob(start_date='-90y', end_date='-18y')  # Generating DOB for patients aged between 18 and 90
        age = calculate_age(dob)
        gender = generate_gender()
        medical_record_number = fake.numerify(text="E14##########")  # Generating medical record number
        
        patient = {
            'FirstName': first_name,
            'LastName': last_name,
            'Name': name,
            'DateOfBirth': dob,
            'Age': age,
            'Gender': gender,
            'MedicalRecordNumber': medical_record_number,
        }
        patients.append(patient)
    return patients

# Example usage
num_patients = 1
patient_data = generate_patient_data(num_patients)
for patient in patient_data:
    print(patient)


{'FirstName': 'Christopher', 'LastName': 'Lopez', 'Name': 'Christopher Lopez', 'DateOfBirth': '1938-02-20', 'Age': 45.0, 'Gender': 'Female', 'MedicalRecordNumber': 'E144740677918'}


In [6]:
# Function to generate random credit card number
def generate_credit_card_number():
    credit_card_number = fake.credit_card_number(card_type='visa')  # Generating Visa card number
    expiration_date = fake.credit_card_expire()  # Generating expiration date
    cvv = fake.credit_card_security_code()  # Generating CVV
    return credit_card_number, expiration_date, cvv

# Function to generate patient credit card data
def generate_patient_credit_card_data(patient_data):
    credit_card_data = []
    for patient in patient_data:
        CreditCardNumber, ExpirationDate, CVV = generate_credit_card_number()
        credit_card = {
            'Name': patient['Name'],
            'MedicalRecordNumber': patient['MedicalRecordNumber'],
            'CreditCardNumber': CreditCardNumber,
            'ExpirationDate': ExpirationDate,
            'CVV': CVV
        }
        credit_card_data.append(credit_card)
    return credit_card_data


credit_card_data = generate_patient_credit_card_data(patient_data)

# Printing generated credit card data for demonstration
print("Credit Card Data:")
for credit_card in credit_card_data:
    print(credit_card)


Credit Card Data:
{'Name': 'Christopher Lopez', 'MedicalRecordNumber': 'E144740677918', 'CreditCardNumber': '4763497495423300', 'ExpirationDate': '01/29', 'CVV': '162'}


In [7]:
# Function to generate fake address in Northwest Arkansas
def generate_nw_arkansas_address():
    # Generate fake address in Northwest Arkansas
    street = fake.street_address()
    city = fake.city()
    state = 'Arkansas'
    zip = fake.zipcode_in_state(state_abbr='AR')
    return street, city, state, zip


# Function to generate patient address
def generate_patient_address(patient_data):
    patient_address = []
    for patient in patient_data:
            street, city, state, zip = generate_nw_arkansas_address()
            address = {
            'Name': patient['Name'],
            'MedicalRecordNumber': patient['MedicalRecordNumber'],
            'street': street,
            'city': city,
            'state': state,
            'zip': zip
        }
            patient_address.append(address)
    return patient_address

patient_address = generate_patient_address(patient_data)

# Printing generated credit card data for demonstration
print("Patient Address Data:")
for address in patient_address:
    print(address)

Patient Address Data:
{'Name': 'Christopher Lopez', 'MedicalRecordNumber': 'E144740677918', 'street': '65607 Johnson Island Apt. 889', 'city': 'South Colleenmouth', 'state': 'Arkansas', 'zip': '71848'}


In [8]:
from faker import Faker
import random
from datetime import datetime, timedelta
import numpy as np

# Function to generate random chief complaint
def generate_chief_complaint(age):
    chief_complaints = {
        'Child': ['Fever', 'Cough', 'Earache', 'Sore throat'],
        'Adult': ['Chest pain', 'Shortness of breath', 'Abdominal pain', 'Headache']
    }
    if age < 18:
        return random.choice(chief_complaints['Child'])
    else:
        return random.choice(chief_complaints['Adult'])

# Function to generate random patient encounters
def generate_patient_encounters(patient_data):
    encounters = []
    for patient in patient_data:
        num_encounters = random.randint(1, 5)  # Generate random number of encounters
        for _ in range(num_encounters):
            dob = datetime.strptime(patient['DateOfBirth'], '%Y-%m-%d')
            start_date = dob + timedelta(days=365 * 2)  # 2 years after birth
            end_date = datetime.today()
            encounter_date = fake.date_time_between_dates(datetime_start=start_date, datetime_end=end_date)
            chief_complaint = generate_chief_complaint(patient['Age'])
            encounter = {
                'PatientName': patient['Name'],
                'MedicalRecordNumber': patient['MedicalRecordNumber'],
                'ChiefComplaint': chief_complaint,
                'EncounterDate': encounter_date.replace(tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
            }
            encounters.append(encounter)
    return encounters

encounters_data = generate_patient_encounters(patient_data)

# Printing generated encounters data for demonstration
print("Encounters Data:")
for encounter in encounters_data:
    print(encounter)

Encounters Data:
{'PatientName': 'Christopher Lopez', 'MedicalRecordNumber': 'E144740677918', 'ChiefComplaint': 'Shortness of breath', 'EncounterDate': '1985-07-12 13:08:35'}
{'PatientName': 'Christopher Lopez', 'MedicalRecordNumber': 'E144740677918', 'ChiefComplaint': 'Chest pain', 'EncounterDate': '1943-01-21 00:26:09'}
{'PatientName': 'Christopher Lopez', 'MedicalRecordNumber': 'E144740677918', 'ChiefComplaint': 'Shortness of breath', 'EncounterDate': '2008-09-21 10:36:59'}


In [12]:
import mysql.connector

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",  # Assuming "Fake_Patient_Info" is your connection name
    user="root",      # Replace "your_username" with your MySQL username
    password="Avengers2@",  # Replace "your_password" with your MySQL password
    database="patient_info"    # Database you want to insert into
)

# Create a cursor object
cursor = conn.cursor()

# Create tables in MySQL database
create_patients_table_query = """
CREATE TABLE IF NOT EXISTS Patients (
    Name VARCHAR(255),
    DateOfBirth DATE,
    Age INT,
    Gender VARCHAR(10),
    MedicalRecordNumber VARCHAR(36)
)
"""

# # Create tables in MySQL database
# create_patients_CC_query = """
# CREATE TABLE IF NOT EXISTS Patients (
#     Name VARCHAR(255),
#     MedicalRecordNumber VARCHAR(36),
#     CreditCardNumber VARCHAR(36),
#     ExpirationDate VARCHAR(36),
#     CVV VARCHAR(36)
    
# )
# """

# # Create tables in MySQL database
# create_patients_address_query = """
# CREATE TABLE IF NOT EXISTS Patients (
#     Name VARCHAR(255),
#     MedicalRecordNumber VARCHAR(36),
#     street VARCHAR(36),
#     city VARCHAR(36),
#     state VARCHAR(36),
#     zip VARCHAR(36
    
# )
# """

create_encounters_table_query = """
CREATE TABLE IF NOT EXISTS Encounters (
    PatientName VARCHAR(255),
    MedicalRecordNumber VARCHAR(36),
    ChiefComplaint VARCHAR(255),
    EncounterDate DATETIME
)
"""

cursor.execute(create_patients_table_query)
cursor.execute(create_encounters_table_query)

# Insert data into Patients table
for patient in patient_data:
    insert_patient_query = """
    INSERT INTO Patients (Name, DateOfBirth, Age, Gender, MedicalRecordNumber, LastHospitalEncounter)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_patient_query, tuple(patient.values()))

# Insert data into Encounters table
for encounter in encounters_data:
    insert_encounter_query = """
    INSERT INTO Encounters (PatientName, MedicalRecordNumber, ChiefComplaint, EncounterDate)
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_encounter_query, (encounter['PatientName'], encounter['MedicalRecordNumber'], encounter['ChiefComplaint'], encounter['EncounterDate']))

# Commit changes and close connection
conn.commit()
cursor.close()
conn.close()


ProgrammingError: Failed processing format-parameters; Python 'float64' cannot be converted to a MySQL type